In [1]:
import json
import yaml
import glob
import os
import numpy as np
import sys

In [2]:
project = "twitter-2024-09-13-20-23-70120e04"
project_dir = os.path.abspath(f"../data/label_studio/{project}")

In [3]:
# Create train.config.yaml
config_d = {"path" : project_dir, "train" : "images", "val" : "images"}
with open(os.path.join(project_dir, "notes.json"), 'r') as infile:
    project_notes = json.load(infile)

id_to_entity = {}
for cat in project_notes['categories']:
    id_to_entity[cat['id']] = cat['name']

config_d["names"] = id_to_entity

config_f = os.path.join(project_dir, "train_config.yaml")
with open(config_f, 'w') as outfile:
    yaml.dump(config_d, outfile)

In [4]:
id_to_entity

{0: 'discover_new_communities',
 1: 'general_button',
 2: 'more_posted',
 3: 'new_posts',
 4: 'partial_tweet',
 5: 'pinned_by_people_you_follow',
 6: 'playing_video',
 7: 'plus_button',
 8: 'quoted_tweet',
 9: 'subscribe_button',
 10: 'tweet',
 11: 'tweet_ad',
 12: 'twitter_bottom_menu',
 13: 'twitter_top_menu'}

In [5]:
# Scrollabel cleanup x labels to full width of image
labels_dir = os.path.join(project_dir, "labels")
for f in os.listdir(labels_dir):
    labels_file_path = os.path.join(labels_dir, f)
    new_label_data = list()
    with open(labels_file_path, 'r') as infile:
        for line in infile.readlines():
            line_s = line.split(" ")
            if len(line_s) != 1:
                if int(line_s[0]) in {0, 4, 5, 6, 10, 11, 12, 13}:
                    new_line = " ".join([line_s[0], str(0.5), line_s[2], str(1), line_s[4]])
                    new_label_data.append(new_line)
                else:
                    new_label_data.append(line)

    with open(labels_file_path, 'w') as outfile:
        for new_line in new_label_data:
            outfile.write(new_line)

# Train

In [6]:
import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [7]:
from ultralytics import YOLO

In [8]:
model = YOLO("yolov8m.pt")

In [9]:
results = model.train(data=config_f, epochs=30, device="mps", rect=True, augment=False, close_mosaic=0, batch=15, scale=0, translate=0)

New https://pypi.org/project/ultralytics/8.2.93 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.91 🚀 Python-3.10.14 torch-2.5.0.dev20240625 MPS (Apple M3 Max)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04/train_config.yaml, epochs=30, time=None, patience=100, batch=15, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agn

train: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04/labels... 75 images, 2 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<00:00, 2866.74it/s]

train: New cache created: /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04/labels.cache
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04/labels.cache... 75 images, 2 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]


Plotting labels to runs/detect/train15/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000556, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00046875), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train15
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING ⚠️ NMS time limit 3.500s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [01:04<00:29, 29.58s/it]

WARNING ⚠️ NMS time limit 2.750s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:28<00:00, 29.52s/it]


                   all         75        284      0.975      0.533      0.636      0.562
discover_new_communities          3          3       0.91          1      0.995      0.907
           more_posted         25         25      0.994       0.56      0.598      0.594
             new_posts          3          3       0.97          1      0.995      0.995
         partial_tweet          1          2          1          0          0          0
pinned_by_people_you_follow          1          1      0.933          1      0.995      0.995
         playing_video          5          6      0.972      0.667      0.667      0.616
           plus_button         15         15      0.993      0.667      0.727      0.708
          quoted_tweet          1          1          1          0          0          0
      subscribe_button          1          1      0.936          1      0.995      0.995
                 tweet         72        189      0.967      0.593      0.675      0.613
              